In [21]:
import pandas as pd
import numpy as np
import os
from save_excel import save_excel
from openpyxl import load_workbook
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Import the cleaned dataset
csv_file_path = '..\\dataset\\international_top_terms_cleaned.csv'
df = pd.read_csv(csv_file_path)
df.head(10)

,country_code,region_name,region_code,week,score,rank,country_name,term,refresh_date
0,FR,Aquitaine,FR-B,2019-09-15,57.0,14,France,Apple,2024-06-14
1,FR,Auvergne,FR-C,2019-09-08,83.0,14,France,Apple,2024-06-14
2,FR,Burgundy,FR-D,2019-09-01,100.0,14,France,Apple,2024-06-14
3,FR,Burgundy,FR-D,2021-09-12,62.0,14,France,Apple,2024-06-14
4,FR,Burgundy,FR-D,2021-11-21,64.0,14,France,Apple,2024-06-14
5,FR,Burgundy,FR-D,2022-01-09,51.0,14,France,Apple,2024-06-14
6,FR,Burgundy,FR-D,2022-11-20,62.0,14,France,Apple,2024-06-14
7,FR,Burgundy,FR-D,2022-12-11,51.0,14,France,Apple,2024-06-14
8,FR,Burgundy,FR-D,2023-07-30,51.0,14,France,Apple,2024-06-14
9,FR,Burgundy,FR-D,2023-11-19,60.0,14,France,Apple,2024-06-14


# Splitting the Data in Accordance to the Country

## Australia
```country_name```

```region_name```

```term```

```score```

```mean``` - rows with same region_name and term will have their mean score computed

In [22]:
basket_Australia = (df[df['country_name'] == "Australia"]
          .groupby(['region_name', 'term'])['score'] 
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Australia)

term                          Alex de Minaur  Alexa Leary  Alice Springs  \
region_name                                                                
Australian Capital Territory       62.000000          0.0      57.000000   
New South Wales                    56.500000          0.0       0.000000   
Northern Territory                100.000000        100.0      77.589744   
Queensland                         75.333333          0.0      54.400000   
South Australia                    72.000000          0.0      64.214286   
Tasmania                           75.000000          0.0      61.000000   
Victoria                           59.000000          0.0      65.666667   
Western Australia                  82.000000          0.0      66.750000   

term                              Apple  Ash Barty  Bacchus Marsh Grammar  \
region_name                                                                 
Australian Capital Territory  65.750000        0.0                    0.0   
New Sout

### Conversion to binary vector

In [23]:
def hot_encode(x):
    if(x <= 0): 
        return 0
    if(x >= 1): 
        return 1

basket_encoded = basket_Australia.map(hot_encode) 
basket_Australia = basket_encoded

print(basket_Australia)

term                          Alex de Minaur  Alexa Leary  Alice Springs  \
region_name                                                                
Australian Capital Territory               1            0              1   
New South Wales                            1            0              0   
Northern Territory                         1            1              1   
Queensland                                 1            0              1   
South Australia                            1            0              1   
Tasmania                                   1            0              1   
Victoria                                   1            0              1   
Western Australia                          1            0              1   

term                          Apple  Ash Barty  Bacchus Marsh Grammar  \
region_name                                                             
Australian Capital Territory      1          0                      0   
New South Wales     

##
```min_support``` - percentage in which the terms occurs together

```min_threshold``` - specifies the minimum confidence or lift for an association rule to be considered significant

```confidence``` - Measures how often the consequent of a rule appears given the antecedent

```lift``` - Indicates the likeliness consequent is to occur when the antecedent is present

In [24]:
frq_items = apriori(basket_Australia.astype('bool'), min_support = 0.9, use_colnames = True)
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

display(rules)
save_excel(rules, "Australia")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Alex de Minaur),(Apple),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
1,(Apple),(Alex de Minaur),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
2,(Alex de Minaur),(Champions League),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
3,(Champions League),(Alex de Minaur),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
4,(Alex de Minaur),(Cricket),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
...,...,...,...,...,...,...,...,...,...,...
6045,(NBA),"(Premier League, Champions League, Full moon, ...",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
6046,(UFC),"(Premier League, Champions League, Full moon, ...",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
6047,(Cricket),"(Premier League, Champions League, Full moon, ...",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
6048,(Alex de Minaur),"(Premier League, Champions League, Full moon, ...",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0


## Argentina

In [25]:
# Top terms in Argentina
basket_Argentina = (df[df['country_name'] == "Argentina"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Argentina)

term                                Alcaraz    Aldosivi  Alphonso Davies  \
region_name                                                                
Autonomous City of Buenos Aires   75.000000   55.000000              0.0   
Buenos Aires Province             50.000000   71.500000              0.0   
Catamarca Province                 0.000000  100.000000              0.0   
Chaco Province                    59.000000    0.000000              0.0   
Chubut Province                   57.500000    0.000000              0.0   
Cordoba                            0.000000   64.333333              0.0   
Corrientes Province               89.000000   57.000000             57.0   
Entre Rios                       100.000000    0.000000             70.0   
Formosa Province                  75.000000    0.000000              0.0   
Jujuy                              0.000000    0.000000              0.0   
La Pampa Province                  0.000000   56.000000              0.0   
La Rioja Pro

In [26]:
basket_encoded_Argentina = basket_Argentina.map(hot_encode) 
basket_Argentina = basket_encoded_Argentina

In [27]:
frq_items = apriori(basket_Argentina.astype('bool'), min_support = 0.5, use_colnames = True)
   
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Argentina")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
37,(Bridgerton),(Independiente),0.625000,0.875,0.625000,1.0,1.142857,0.078125,inf,0.333333
52,(Copa america),(San Antonio),0.500000,0.875,0.500000,1.0,1.142857,0.062500,inf,0.250000
93,"(Alcaraz, Bridgerton)",(Independiente),0.500000,0.875,0.500000,1.0,1.142857,0.062500,inf,0.250000
355,"(Futbol argentino, Bridgerton)",(Independiente),0.541667,0.875,0.541667,1.0,1.142857,0.067708,inf,0.272727
361,"(Bridgerton, Primera división)",(Independiente),0.541667,0.875,0.541667,1.0,1.142857,0.067708,inf,0.272727
...,...,...,...,...,...,...,...,...,...,...
2759,(River),"(TN, Independiente, Racing, San Antonio)",1.000000,0.500,0.500000,0.5,1.000000,0.000000,1.0,0.000000
2843,(River),"(Racing, Aldosivi, San Antonio, Primera divisi...",1.000000,0.500,0.500000,0.5,1.000000,0.000000,1.0,0.000000
2998,(River),"(Racing, TN, Primera división, Futbol argentin...",1.000000,0.500,0.500000,0.5,1.000000,0.000000,1.0,0.000000
3040,(River),"(San Antonio, TN, Primera división, Futbol arg...",1.000000,0.500,0.500000,0.5,1.000000,0.000000,1.0,0.000000


## Philippines

In [28]:
# Top terms in Philippines
basket_Philippines = (df[df['country_name'] == "Philippines"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Philippines)

term                                     19     Access  Angelina Jolie  \
region_name                                                              
Autonomous Region in Muslim Mindanao    0.0  59.750000        0.000000   
Bicol                                  67.0  65.137931        0.000000   
Cagayan Valley                          0.0  63.428571        0.000000   
Calabarzon                             72.0  70.625000        0.000000   
Caraga                                100.0  66.866667       92.000000   
Central Luzon                          96.0  67.862069        0.000000   
Central Visayas                        58.0  69.111111       59.000000   
Cordillera Administrative Region        0.0  69.312500      100.000000   
Davao Region                           56.5  65.172414       76.333333   
Eastern Visayas                        67.0  59.500000        0.000000   
Ilocos Region                           0.0  59.772727       76.000000   
MIMAROPA                              

In [29]:
basket_encoded_Philippines = basket_Philippines.map(hot_encode) 
basket_Philippines = basket_encoded_Philippines

print(basket_Philippines)

term                                  19  Access  Angelina Jolie  Anouk Aimée  \
region_name                                                                     
Autonomous Region in Muslim Mindanao   0       1               0            0   
Bicol                                  1       1               0            0   
Cagayan Valley                         0       1               0            0   
Calabarzon                             1       1               0            0   
Caraga                                 1       1               1            0   
Central Luzon                          1       1               0            0   
Central Visayas                        1       1               1            1   
Cordillera Administrative Region       0       1               1            0   
Davao Region                           1       1               1            0   
Eastern Visayas                        1       1               0            0   
Ilocos Region               

In [30]:
frq_items = apriori(basket_Philippines.astype('bool'), min_support = 0.8, use_colnames = True)

print("After frequent Items")
 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Philippines")

After frequent Items


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ESPN NBA),(Access),0.882353,1.000000,0.882353,1.000000,1.0,0.0,inf,0.0
2,(Jose Rizal),(Access),0.823529,1.000000,0.823529,1.000000,1.0,0.0,inf,0.0
4,(June),(Access),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf,0.0
5,(Access),(June),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf,0.0
6,(NBA),(Access),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf,0.0
...,...,...,...,...,...,...,...,...,...,...
1440,(June),"(NBA, NBA games, Sports, Access, UFC)",1.000000,0.823529,0.823529,0.823529,1.0,0.0,1.0,0.0
1441,(NBA),"(June, NBA games, Sports, Access, UFC)",1.000000,0.823529,0.823529,0.823529,1.0,0.0,1.0,0.0
1442,(NBA games),"(June, NBA, Sports, Access, UFC)",1.000000,0.823529,0.823529,0.823529,1.0,0.0,1.0,0.0
1443,(Sports),"(June, NBA, NBA games, Access, UFC)",1.000000,0.823529,0.823529,0.823529,1.0,0.0,1.0,0.0


## South Korea

In [31]:
# Top terms in South Korea
basket_SouthKorea = (df[df['country_name'] == "South Korea"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_SouthKorea)

term                 CPI        NBA   WWDC    강민구    고준희    권경애    김계란    김레아  \
region_name                                                                     
Busan                0.0  63.333333    0.0    0.0    0.0    0.0   83.5    0.0   
Chungcheongbuk-do    0.0  63.666667    0.0    0.0    0.0    0.0    0.0   80.0   
Chungcheongnam-do   77.0  60.571429    0.0  100.0  100.0    0.0   56.0    0.0   
Daegu                0.0  59.000000    0.0    0.0    0.0    0.0   52.0   94.0   
Daejeon            100.0  55.500000    0.0    0.0    0.0  100.0    0.0    0.0   
Gangwon-do           0.0  54.000000    0.0    0.0    0.0    0.0  100.0    0.0   
Gwangju            100.0  62.000000    0.0    0.0    0.0    0.0   84.0  100.0   
Gyeonggi-do         77.5  61.571429    0.0  100.0    0.0    0.0   64.5    0.0   
Gyeongsangbuk-do     0.0  64.894737    0.0    0.0    0.0    0.0   61.5    0.0   
Gyeongsangnam-do     0.0  54.000000  100.0  100.0    0.0    0.0   67.0    0.0   
Incheon             59.0  54

In [32]:
basket_encoded_SouthKorea = basket_SouthKorea.map(hot_encode) 
basket_SouthKorea = basket_encoded_SouthKorea

print(basket_SouthKorea)

term               CPI  NBA  WWDC  강민구  고준희  권경애  김계란  김레아  김미성  노스트라다무스 예언  \
region_name                                                                   
Busan                0    1     0    0    0    0    1    0    0           0   
Chungcheongbuk-do    0    1     0    0    0    0    0    1    0           0   
Chungcheongnam-do    1    1     0    1    1    0    1    0    0           1   
Daegu                0    1     0    0    0    0    1    1    0           1   
Daejeon              1    1     0    0    0    1    0    0    0           0   
Gangwon-do           0    1     0    0    0    0    1    0    0           1   
Gwangju              1    1     0    0    0    0    1    1    0           0   
Gyeonggi-do          1    1     0    1    0    0    1    0    0           0   
Gyeongsangbuk-do     0    1     0    0    0    0    1    0    0           0   
Gyeongsangnam-do     0    1     1    1    0    0    1    0    0           0   
Incheon              1    1     0    0    1    0    

In [33]:
frq_items = apriori(basket_SouthKorea.astype('bool'), min_support = 0.8, use_colnames = True)

print("After frequent Items")
 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "South_Korea")

After frequent Items


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(라이엇),(NBA),0.9375,1.0000,0.9375,1.0000,1.0,0.0,inf,0.0
3,(리그오브레전드),(NBA),0.8750,1.0000,0.8750,1.0000,1.0,0.0,inf,0.0
4,(메이플스토리),(NBA),1.0000,1.0000,1.0000,1.0000,1.0,0.0,inf,0.0
5,(NBA),(메이플스토리),1.0000,1.0000,1.0000,1.0000,1.0,0.0,inf,0.0
6,(NBA),(엔비디아),1.0000,1.0000,1.0000,1.0000,1.0,0.0,inf,0.0
...,...,...,...,...,...,...,...,...,...,...
435,"(메이플스토리, 엔비디아)","(NBA, 라이엇, 한국, 리그오브레전드)",1.0000,0.8125,0.8125,0.8125,1.0,0.0,1.0,0.0
436,(한국),"(리그오브레전드, NBA, 라이엇, 메이플스토리, 엔비디아)",1.0000,0.8125,0.8125,0.8125,1.0,0.0,1.0,0.0
437,(NBA),"(한국, 리그오브레전드, 라이엇, 메이플스토리, 엔비디아)",1.0000,0.8125,0.8125,0.8125,1.0,0.0,1.0,0.0
438,(메이플스토리),"(한국, 리그오브레전드, NBA, 라이엇, 엔비디아)",1.0000,0.8125,0.8125,0.8125,1.0,0.0,1.0,0.0


## Finland

In [34]:
# Top terms in Finland
basket_Finland = (df[df['country_name'] == "Finland"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Finland)

term                   Adam Lambert  Albania       Alko      Apple  Barkov  \
region_name                                                                  
Central Finland                 0.0    100.0  59.333333  59.500000     0.0   
Central Ostrobothnia            0.0      0.0  63.000000  58.000000     0.0   
Kainuu                        100.0     94.0  50.000000  82.500000     0.0   
Kymenlaakso                     0.0      0.0  52.500000  61.000000     0.0   
Lapland                       100.0      0.0  69.250000  57.333333     0.0   
North Karelia                   0.0    100.0  57.000000  59.200000     0.0   
Northern Ostrobothnia          76.0     88.0  73.000000  64.166667   100.0   
Northern Savonia                0.0     75.0   0.000000  62.100000     0.0   
Ostrobothnia                    0.0      0.0  62.000000  57.900000     0.0   
Pirkanmaa                       0.0      0.0  71.500000  54.166667     0.0   
Päijänne Tavastia               0.0      0.0  61.000000  53.7142

In [35]:
basket_encoded_Finland = basket_Finland.map(hot_encode) 
basket_Finland = basket_encoded_Finland

print(basket_Finland)

term                   Adam Lambert  Albania  Alko  Apple  Barkov  Bellingham  \
region_name                                                                     
Central Finland                   0        1     1      1       0           0   
Central Ostrobothnia              0        0     1      1       0           0   
Kainuu                            1        1     1      1       0           1   
Kymenlaakso                       0        0     1      1       0           1   
Lapland                           1        0     1      1       0           0   
North Karelia                     0        1     1      1       0           0   
Northern Ostrobothnia             1        1     1      1       1           0   
Northern Savonia                  0        1     0      1       0           0   
Ostrobothnia                      0        0     1      1       0           0   
Pirkanmaa                         0        0     1      1       0           0   
Päijänne Tavastia           

In [36]:
frq_items = apriori(basket_Finland.astype('bool'), min_support = 0.3, use_colnames = True)
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Finland")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1103,"(Bridgerton, Billie Eilish)","(Nvidia, UFC)",0.388889,0.500000,0.388889,1.000000,2.0,0.194444,inf,0.818182
1423,"(Alko, Bridgerton, Billie Eilish)","(Nvidia, UFC)",0.333333,0.500000,0.333333,1.000000,2.0,0.166667,inf,0.750000
1452,"(Apple, Bridgerton, Billie Eilish)","(Nvidia, UFC)",0.388889,0.500000,0.388889,1.000000,2.0,0.194444,inf,0.818182
1459,"(Bridgerton, Billie Eilish)","(Apple, Nvidia, UFC)",0.388889,0.500000,0.388889,1.000000,2.0,0.194444,inf,0.818182
1482,"(Apple, Alko, Bridgerton, Billie Eilish)","(Nvidia, UFC)",0.333333,0.500000,0.333333,1.000000,2.0,0.166667,inf,0.750000
...,...,...,...,...,...,...,...,...,...,...
1331,(Apple),"(Ronaldo, Real Madrid, Alko, Nvidia)",1.000000,0.333333,0.333333,0.333333,1.0,0.000000,1.0,0.000000
1353,(Apple),"(Ronaldo, Alko, Nvidia, Star Wars)",1.000000,0.333333,0.333333,0.333333,1.0,0.000000,1.0,0.000000
1383,(Apple),"(Ronaldo, Alko, Nvidia, UFC)",1.000000,0.333333,0.333333,0.333333,1.0,0.000000,1.0,0.000000
1413,(Apple),"(Alko, Nvidia, Star Wars, UFC)",1.000000,0.333333,0.333333,0.333333,1.0,0.000000,1.0,0.000000


## United Kingdom

In [37]:
# Top terms in United Kingdom
basket_UK = (df[df['country_name'] == "United Kingdom"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_UK)

term              Alan Shearer  Albania      Apple  BGT final  Bath Rugby  \
region_name                                                                 
England                    0.0      0.0  58.176471       74.0        50.0   
Northern Ireland          73.5      0.0  56.909091        0.0         0.0   
Scotland                   0.0      0.0  62.238095        0.0        59.5   
Wales                     58.0     74.0  56.444444        0.0        67.0   

term              Billings  Billy Harris  Bridgerton  Britain's Got Talent  \
region_name                                                                  
England                0.0         100.0         0.0                   0.0   
Northern Ireland      94.0           0.0         0.0                   0.0   
Scotland               0.0           0.0        61.0                 100.0   
Wales                 64.0           0.0         0.0                   0.0   

term              Carlo Ancelotti  ...  Sara Cox   Scotland  Sloveni

In [38]:
basket_encoded_UK = basket_UK.map(hot_encode) 
basket_UK = basket_encoded_UK

print(basket_UK)

term              Alan Shearer  Albania  Apple  BGT final  Bath Rugby  \
region_name                                                             
England                      0        0      1          1           1   
Northern Ireland             1        0      1          0           0   
Scotland                     0        0      1          0           1   
Wales                        1        1      1          0           1   

term              Billings  Billy Harris  Bridgerton  Britain's Got Talent  \
region_name                                                                  
England                  0             1           0                     0   
Northern Ireland         1             0           0                     0   
Scotland                 0             0           1                     1   
Wales                    1             0           0                     0   

term              Carlo Ancelotti  ...  Sara Cox  Scotland  Slovenia  Spain  \
region_name  

In [39]:
frq_items = apriori(basket_UK.astype('bool'), min_support = 0.8, use_colnames = True)
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "United_Kingdom")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(England football),(Apple),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
1,(Apple),(England football),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
2,(Gary Glitter),(Apple),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
3,(Apple),(Gary Glitter),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
4,(Love Island),(Apple),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
...,...,...,...,...,...,...,...,...,...,...
597,(Apple),"(Love Island, England football, UCL, Gary Glit...",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
598,(England football),"(Love Island, Apple, UCL, Gary Glitter, UFC)",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
599,(UCL),"(Love Island, Apple, England football, Gary Gl...",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
600,(Gary Glitter),"(Love Island, Apple, England football, UCL, UFC)",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0


In [40]:
basket_SF = (df[df['country_name'] == "South Africa"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_SF)


term               Apple  Argentina vs Canada  Bangladesh vs Netherlands  \
region_name                                                                
Eastern Cape   62.692308                  0.0                        0.0   
Free State     60.250000                 81.0                        0.0   
Gauteng        64.727273                  0.0                        0.0   
KwaZulu-Natal  64.923077                  0.0                        0.0   
Limpopo        59.074074                  0.0                        0.0   
Mpumalanga     58.285714                  0.0                       85.0   
North West     61.071429                  0.0                        0.0   
Northern Cape  63.500000                  0.0                        0.0   
Western Cape   69.081081                  0.0                        0.0   

term           Baroka FC  Bridgerton  Charley Hull  Coco Gauff  \
region_name                                                      
Eastern Cape   51.000000   87.0

In [41]:
basket_encoded_SF = basket_SF.map(hot_encode) 
basket_SF = basket_encoded_SF

print(basket_SF)

term           Apple  Argentina vs Canada  Bangladesh vs Netherlands  \
region_name                                                            
Eastern Cape       1                    0                          0   
Free State         1                    1                          0   
Gauteng            1                    0                          0   
KwaZulu-Natal      1                    0                          0   
Limpopo            1                    0                          0   
Mpumalanga         1                    0                          1   
North West         1                    0                          0   
Northern Cape      1                    0                          0   
Western Cape       1                    0                          0   

term           Baroka FC  Bridgerton  Charley Hull  Coco Gauff  \
region_name                                                      
Eastern Cape           1           1             0           0   
Free Stat

In [42]:
frq_items = apriori(basket_SF.astype('bool'), min_support = 0.5, use_colnames = True)
 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "South_Africa")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
358,"(Malawi, Cricket World Cup)",(Lesotho),0.555556,0.555556,0.555556,1.000000,1.8,0.246914,inf,1.0
359,(Lesotho),"(Malawi, Cricket World Cup)",0.555556,0.555556,0.555556,1.000000,1.8,0.246914,inf,1.0
370,"(Poetry, Cricket World Cup)",(Lesotho),0.555556,0.555556,0.555556,1.000000,1.8,0.246914,inf,1.0
371,(Lesotho),"(Poetry, Cricket World Cup)",0.555556,0.555556,0.555556,1.000000,1.8,0.246914,inf,1.0
727,"(Apple, Malawi, Cricket World Cup)",(Lesotho),0.555556,0.555556,0.555556,1.000000,1.8,0.246914,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
4409,(Apple),"(Poetry, Malawi, Orlando Pirates, Lesotho, Sou...",1.000000,0.555556,0.555556,0.555556,1.0,0.000000,1.0,0.0
4471,(Apple),"(Poetry, Malawi, Orlando Pirates, T20 World Cu...",1.000000,0.555556,0.555556,0.555556,1.0,0.000000,1.0,0.0
4529,(Apple),"(Poetry, The Boys, Malawi, Orlando Pirates, So...",1.000000,0.555556,0.555556,0.555556,1.0,0.000000,1.0,0.0
4779,(Apple),"(Poetry, Malawi, Cricket World Cup, Orlando Pi...",1.000000,0.555556,0.555556,0.555556,1.0,0.000000,1.0,0.0


In [43]:
basket_Canada = (df[df['country_name'] == "Canada"]
          .groupby(['region_name', 'term'])['score']
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Canada)

term                       Alex Caruso   Billings  Bobrovsky  Bridgerton  \
region_name                                                                
Alberta                          100.0  56.500000        0.0         0.0   
British Columbia                   0.0   0.000000      100.0        57.0   
Manitoba                           0.0  84.000000        0.0         0.0   
New Brunswick                      0.0   0.000000        0.0         0.0   
Newfoundland and Labrador          0.0   0.000000        0.0        76.5   
Northwest Territories              0.0   0.000000        0.0         0.0   
Nova Scotia                        0.0   0.000000        0.0        71.0   
Nunavut                            0.0   0.000000      100.0         0.0   
Ontario                           54.0  68.400000       86.0         0.0   
Prince Edward Island               0.0   0.000000       94.0        71.5   
Québec                            63.0  58.363636        0.0        85.0   
Saskatchewan

In [44]:
basket_encoded_Canada = basket_Canada.map(hot_encode) 
basket_Canada = basket_encoded_Canada

print(basket_Canada)

term                       Alex Caruso  Billings  Bobrovsky  Bridgerton  \
region_name                                                               
Alberta                              1         1          0           0   
British Columbia                     0         0          1           1   
Manitoba                             0         1          0           0   
New Brunswick                        0         0          0           0   
Newfoundland and Labrador            0         0          0           1   
Northwest Territories                0         0          0           0   
Nova Scotia                          0         0          0           1   
Nunavut                              0         0          1           0   
Ontario                              1         1          1           0   
Prince Edward Island                 0         0          1           1   
Québec                               1         1          0           1   
Saskatchewan             

In [45]:
frq_items = apriori(basket_Canada.astype('bool'), min_support = 0.4, use_colnames = True)
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

display(rules)
save_excel(rules, "Canada")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
426,"(Buffalo, PGA)","(The Boys, LPGA)",0.461538,0.538462,0.461538,1.000000,1.857143,0.213018,inf,0.857143
625,"(Oilers, The Boys)","(PGA, Rangers)",0.461538,0.538462,0.461538,1.000000,1.857143,0.213018,inf,0.857143
673,"(Buffalo, Et, PGA)","(The Boys, LPGA)",0.461538,0.538462,0.461538,1.000000,1.857143,0.213018,inf,0.857143
684,"(Buffalo, PGA)","(Et, The Boys, LPGA)",0.461538,0.538462,0.461538,1.000000,1.857143,0.213018,inf,0.857143
728,"(Et, Oilers, The Boys)","(Rangers, PGA)",0.461538,0.538462,0.461538,1.000000,1.857143,0.213018,inf,0.857143
...,...,...,...,...,...,...,...,...,...,...
586,(Et),"(Oilers, The Boys, Rangers)",1.000000,0.461538,0.461538,0.461538,1.000000,0.000000,1.0,0.000000
658,(Et),"(Oilers, Buffalo, LPGA, Rangers)",1.000000,0.461538,0.461538,0.461538,1.000000,0.000000,1.0,0.000000
688,(Et),"(Buffalo, The Boys, LPGA, PGA)",1.000000,0.461538,0.461538,0.461538,1.000000,0.000000,1.0,0.000000
718,(Et),"(Rangers, The Boys, LPGA, PGA)",1.000000,0.461538,0.461538,0.461538,1.000000,0.000000,1.0,0.000000
